In [ ]:
!pip install langchain langchain_experimental
!pip install watermark
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.5 MB/s eta 0:00:00


In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import OpenAI

In [ ]:
import pandas as pd

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
url = "https://raw.githubusercontent.com/adamerose/datasets/master/titanic.csv"
df = pd.read_csv(url)
print(df.shape)
df.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### Single DataFrame

In [ ]:
llm = OpenAI()

In [ ]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True)

In [ ]:
agent.run("How many rows are there")



> Entering new AgentExecutor chain...
Thought: I need to find the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [ ]:
agent.run("How many people have age less than 18")



> Entering new AgentExecutor chain...
Thought: We can use a conditional statement to filter out rows where the age is less than 18 and then count the number of rows.
Action: python_repl_ast
Action Input: len(df[df['age'] < 18])113113 people have age less than 18.
Final Answer: 113

> Finished chain.


'113'

### Multi DataFrame

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


In [ ]:
df1 = df.copy()

In [ ]:
df1["age"] = df1["age"].fillna(df1["age"].mean())

In [ ]:
agent = create_pandas_dataframe_agent(llm, [df, df1], verbose=True, allow_dangerous_code=True)

In [ ]:
agent.run("How many rows in the age column are different")



> Entering new AgentExecutor chain...
Thought: We need to find the number of unique values in the 'age' column
Action: python_repl_ast
Action Input: len(df['age'].unique())NameError: name 'df' is not definedWe need to specify which dataframe we are using
Action: python_repl_ast
Action Input: len(df1['age'].unique())8989 rows have unique values in the 'age' column for dataframe 1
Action: python_repl_ast
Action Input: len(df2['age'].unique())8989 rows have unique values in the 'age' column for dataframe 2
Final Answer: 89

> Finished chain.


'89'

In [ ]:
df2 = df1.copy()
df2["Age_Multiplied"] = df1["age"]*2

In [ ]:
df2.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,Age_Multiplied
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,44.0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,76.0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,52.0
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,70.0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,70.0


In [ ]:
agent = create_pandas_dataframe_agent(llm, [df, df1, df2], verbose=True, allow_dangerous_code=True)

In [ ]:
agent.run("Are the number of columns same in all the dataframe")



> Entering new AgentExecutor chain...
Thought: I need to find the number of columns in each dataframe and compare them
Action: python_repl_ast
Action Input: len(df1.columns)1515 columns in df1
Action: python_repl_ast
Action Input: len(df2.columns)1515 columns in df2
Action: python_repl_ast
Action Input: len(df3.columns)1616 columns in df3
Action: python_repl_ast
Action Input: 15 == 15 == 16False15 columns in df1 and df2 are the same, but df3 has an extra column
Final Answer: No, the number of columns are not the same in all dataframes.

> Finished chain.


'No, the number of columns are not the same in all dataframes.'